<a href="https://colab.research.google.com/github/Servetvrll/kredi_risk_siniflandirma/blob/main/Kredi_riski_analizi_Son.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# veri setine ilgili linkten ulaşabilirsiniz = https://www.kaggle.com/datasets/laotse/credit-risk-dataset
# Veri setini Google Drive'dan Colab ortamına yüklüyoruz.
# Eğer lokalde çalışıyorsanız, dosya yolunu güncelleyin.
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/credit_risk_dataset/credit_risk_dataset.csv")


# Veri setinin ilk 5 satırını göster
print(data.head())
print("---------------------")

# veri seti hakkında genel bilgileri (sütun tipleri,eksik değerler) gösterir
print(data.info())
print("---------------------")

# sayısal sütunlar için istatiksel özet verir
print(data.describe())
print("---------------------")

# Her sütundaki eksik değerlerin sayısını gösterir
print(data.isnull().sum())
print("---------------------")


In [ ]:
# 'person_emp_length' sütunundaki 30 yıldan yüksek değerlerin sayısını verir
person_emp_aykirideger = (data["person_emp_length"] > 30).sum()
print(f"'person_emp_length' sütununda 30 yıldan fazla çalışma süresine sahip {person_emp_aykirideger} kayıt var.")
print("---------------------")

# 'person_age' sütununda 80'den yüksek değerlerin sayısını ve değerleri verir
print("'person_age' sütunundaki 80 yaşından büyük kayıtlar:")
print(data[data["person_age"] > 80]["person_age"])
print("---------------------")

# Anormal dereceden büyük olan (123 gibi) çalışma süreleri muhtemelen hatalı girilmiş.
# Bunların yıllara çevrilmiş hatalı aylık veya başka bir birim olabileceği varsayarak
# daha makul bir değere (örneğin 10 yıl) eşitlenmesi.
data.loc[[0, 210], "person_emp_length"] = [10, 10]

# 'person_age' sütunundaki 80 yaşından büyük aykırı değerleri kaldırma.
# Bu değerler, gerçekçi olmayan yaşları temsil ettiğinden veri bütünlüğü için silinmiştir.
data.drop(data[data["person_age"] > 80].index, inplace=True)
print("80 yaşından büyük kayıtlar silindikten sonraki sayısal özet:")
print(data.describe())
print("---------------------")

In [ ]:
# 'person_emp_length' sütunundaki eksik değerleri medyan ile doldurma
# Çalışma süresinin 0 ile 20 yıl arasında değiştiği göz önüne alındığında,
# eksik değerlerin (muhtemelen çalışmama veya belirsizlik durumu) medyan ile doldurulması,
# mevcut dağılımı koruyarak aykırı değerlerin etkisini azaltır.
data["person_emp_length"].fillna(data["person_emp_length"].median(), inplace=True)

# 'loan_int_rate' sütunundaki eksik değerleri 'loan_grade' gruplarına göre medyan ile doldurma
# Faiz oranının kredi notu (loan_grade) ile ilişkili olduğu varsayımıyla,
# her bir kredi notu grubundaki medyan faiz oranı ile doldurma daha mantıklı bir yaklaşım sunar.
data['loan_int_rate'] = data.groupby('loan_grade')['loan_int_rate'].transform(
    lambda x: x.fillna(x.median())
)

# Eksik değerlerin kalıp kalmadığını kontrol etme
print("Eksik değer doldurulduktan sonraki durum:")
print(data.isnull().sum())
print("---------------------")

In [ ]:
# Kategorik sütunlardaki her bir sınıfın dağılımını (sayısını) kontrol etme
# Bu, kategorik özelliklerin dengesini ve her bir sınıfın veri setindeki temsilini anlamaya yardımcı olur.
print("Kategori Sayıları - loan_grade:")
print(data['loan_grade'].value_counts())
print("\nKategori Sayıları - person_home_ownership:")
print(data['person_home_ownership'].value_counts())
print("\nKategori Sayıları - loan_intent:")
print(data['loan_intent'].value_counts())
print("\nKategori Sayıları - cb_person_default_on_file:")
print(data['cb_person_default_on_file'].value_counts())
print("---------------------")

# Kategorik özellikleri sayısal formata dönüştürme (Label Encoding ve One-Hot Encoding)
from sklearn.preprocessing import LabelEncoder

# Veri setindeki 'object' (kategorik) tipindeki sütunları seçme
kategorik_sutunlar = data.select_dtypes(include=['object']).columns
print("Dönüştürülecek Kategorik Sütunlar:")
print(kategorik_sutunlar)

for col in kategorik_sutunlar:
    if data[col].nunique() == 2:
        # İki sınıfı olan kategorik sütunlar için Label Encoding (ikili dönüşüm)
        # Örn: Yes / No -> 1 / 0 gibi değerler
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
    else:
        # İkiden fazla sınıfı olan kategorik sütunlar için One-Hot Encoding
        # Yeni sütunlar oluşturarak her bir sınıfı ikili (0 veya 1) olarak temsil eder.
        # 'drop_first=False' ile tüm kategoriler tutulur.
        data = pd.get_dummies(data, columns=[col], drop_first=False)

print("\nKategorik sütunlar dönüştürüldükten sonra veri setinin ilk 5 satırı:")
print(data.head())
print("---------------------")

In [ ]:
# Sayısal sütunlardaki aykırı değerleri tespit etme ve görselleştirme
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
print("IQR Yöntemiyle Aykırı Değer Tespiti:")

for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    print(f"{col}: {len(outliers)} aykırı değer")
print("---------------------")

# Her bir sayısal sütun için Boxplot görselleştirme (aykırı değerleri görmek için)
print("Sayısal Sütunların Boxplot Görselleştirmesi:")
for col in numerical_cols:
    plt.figure(figsize=(6, 1.5))
    sns.boxplot(x=data[col])
    plt.title(f"{col} için Boxplot")
    plt.show()
print("---------------------")

# Z-Skoru Yöntemiyle Aykırı Değer Tespiti
from scipy.stats import zscore

z_scores = data[numerical_cols].apply(zscore)
outliers_zscore = (abs(z_scores) > 3).sum()
print("Z-Skoru Yöntemiyle Aykırı Değer Sayıları (Z-skoru > 3):")
print(outliers_zscore)
print("---------------------")

# Belirlenen sütunlardaki aykırı değerleri IQR yöntemine göre kırpma (clipping)
# Bu yöntem, aykırı değerleri belirli bir alt ve üst sınıra çekerek modelin performansını artırabilir.
clip_columns = [
    'person_age', 'person_income', 'person_emp_length',
    'loan_amnt', 'loan_int_rate', 'loan_percent_income',
    'cb_person_cred_hist_length'
]

print("Aykırı Değerleri Kırpma İşlemi (IQR Yöntemiyle):")
for col in clip_columns:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data[col] = data[col].clip(lower_bound, upper_bound)
    print(f"'{col}' sütunundaki aykırı değerler [{lower_bound:.2f}, {upper_bound:.2f}] aralığına kırpıldı.")
print("---------------------")

# Kırpma sonrası 'loan_status'a göre 'person_income' dağılımı görselleştirmesi
print("'loan_status'a göre 'person_income' dağılımı (kırpma sonrası):")
sns.boxplot(x='loan_status', y='person_income', data=data)
plt.title("Kredi Durumuna Göre Gelir Dağılımı (Aykırı Değerler Kırpıldıktan Sonra)")
plt.show()
print("---------------------")

In [ ]:
# StandardScaler, veriyi ortalaması 0 ve standart sapması 1 olacak şekilde dönüştürür.
from sklearn.preprocessing import StandardScaler

# Kırpılan sütunlar ölçeklendirme için de kullanıldı
scale_columns = clip_columns

# StandardScaler nesnesini başlat
scaler = StandardScaler()

# Belirlenen sütunlara standardizasyon uygulama
data[scale_columns] = scaler.fit_transform(data[scale_columns])

print("Standardizasyon Yapıldıktan Sonraki Veri Seti (İlk 5 Satır):")
print(data[scale_columns].head())
print("---------------------")

# Ölçeklendirme ve diğer önişleme adımlarından sonra hedef değişken dağılımını tekrar kontrol etme
sns.countplot(x='loan_status', data=data)
plt.title("Kredi Durumlarının Dağılımı (Önişleme Sonrası)")
plt.xlabel("Kredi Durumu (0: Onaylandı, 1: Reddedildi)")
plt.ylabel("Sayı")
plt.show()
print("---------------------")

# Önişleme sonrası özellikler arasındaki korelasyonu (ilişkiyi) gösteren ısı haritası
plt.figure(figsize=(16, 12))
sns.heatmap(data.corr(), annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)
plt.title("Önişleme Sonrası Korelasyon Matrisi")
plt.show()
print("---------------------")

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# One-Hot Encoding sonrası oluşan 'bool' tipi sütunları int'e dönüştürme
for col in data.select_dtypes('bool').columns:
    data[col] = data[col].astype(int)

print("Dönüştürülmüş Veri Setinin İlk 5 Satırı (Bool -> Int):\n")
print(data.head())
print("---------------------")

# Özellikleri (x) ve hedef değişkeni (y) ayırma
x = data.drop("loan_status", axis=1) # "loan_status" sütunu hedef değişkendir
y = data["loan_status"]

# Eğitim ve test setlerine ayırma
# test_size=0.2: Verinin %20'si test seti olarak ayrıldı
# random_state=42: Bölme işleminin tekrarlanabilir olmasını sağlar
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Sınıf dengesizliğini gidermek için SMOTE uygulama
# Hedef değişkendeki (loan_status) azınlık sınıfının örneklerini sentetik olarak artırır.
# Bu modelin azınlık sınıfını daha iyi öğrenmesine yardımcı olur.
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)

print("SMOTE öncesi eğitim seti sınıf dağılımı:")
print(y_train.value_counts())
print("\nSMOTE sonrası eğitim seti sınıf dağılımı:")
print(pd.Series(y_resampled).value_counts())
print("---------------------")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Kullanılacak modeller
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

results = {}
print("Çapraz Doğrulama Sonuçları (F1 Skoru):")
for name, model in models.items():
    # Her bir model için 5 katlı çapraz doğrulama uygula
    # 'f1' scoring, dengesiz sınıflar için uygun bir metrik.
    scores = cross_val_score(model, x_resampled, y_resampled, cv=5, scoring='f1', n_jobs=-1)
    results[name] = scores
    print(f"{name} - F1 Ortalama Skoru: {scores.mean():.4f} | Standart Sapma: {scores.std():.4f}")
print("---------------------")

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay
from xgboost import XGBClassifier

print("Seçilen Modellerin Test Seti Performansı:\n")

# XGBoost Modelini Eğitme ve Değerlendirme
print("---------- XGBoost Modeli ----------")
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(x_resampled, y_resampled)
y_pred_xgb = xgb_model.predict(x_test)
y_prob_xgb = xgb_model.predict_proba(x_test)[:, 1]

print("XGBoost - Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_xgb))
print("\nXGBoost - Karmaşıklık Matrisi:\n", confusion_matrix(y_test, y_pred_xgb))
print("---------------------")

# Random Forest Modelini Eğitme ve Değerlendirme
print("---------- Random Forest Modeli ----------")
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(x_resampled, y_resampled)
y_pred_rf = rf_model.predict(x_test)
y_prob_rf = rf_model.predict_proba(x_test)[:, 1]

print("Random Forest - Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_rf))
print("\nRandom Forest - Karmaşıklık Matrisi:\n", confusion_matrix(y_test, y_pred_rf))
print("---------------------")

# ROC AUC Skoru ve ROC Eğrisi Görselleştirmesi
# ROC AUC: Modelin pozitif sınıfı doğru tahmin etme yeteneğinin bir ölçüsüdür.
# ROC Eğrisi: Farklı sınıflandırma eşiklerinde True Positive Rate (TPR) ile False Positive Rate (FPR) arasındaki ilişkiyi gösterir.

# ROC AUC Skorlarını Hesaplama
auc_xgb = roc_auc_score(y_test, y_prob_xgb)
auc_rf = roc_auc_score(y_test, y_prob_rf)

print(f"XGBoost ROC AUC: {auc_xgb:.4f}")
print(f"Random Forest ROC AUC: {auc_rf:.4f}")
print("---------------------")

# ROC Eğrilerini Çizme ve Karşılaştırma
print("ROC Eğrisi Karşılaştırması:")
plt.figure(figsize=(8, 6))

# XGBoost için ROC eğrisi
RocCurveDisplay.from_predictions(y_test, y_prob_xgb, name="XGBoost", ax=plt.gca())

# Random Forest için ROC eğrisi
RocCurveDisplay.from_predictions(y_test, y_prob_rf, name="Random Forest", ax=plt.gca())

plt.plot([0, 1], [0, 1], "k--", label="Rastgele Sınıflandırıcı")
plt.title("ROC Eğrisi Karşılaştırması")
plt.xlabel("Yanlış Pozitif Oranı (FPR)")
plt.ylabel("Doğru Pozitif Oranı (TPR)")
plt.grid(True)
plt.legend()
plt.show()
print("---------------------")